In [2]:
from bs4 import BeautifulSoup
import requests
import time
import os
from pprint import pprint
import re
import codecs
from pymongo import MongoClient

## Question 1

In [6]:
headers = {
        "user-agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/74.0.3729.169 Safari/537.36" ,
        "authority": "www.tagesschau.de",
        "method": "GET",
        "path":"/",
        "scheme":"https",
        "accept": "text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.9",
        "accept-encoding": "gzip, deflate, br",
        "accept-language": "en-US,en;q=0.9,de;q=0.8",
        "cache-control": "max-age=0",
        "cookie": "atuserid=%7B%22name%22%3A%22atuserid%22%2C%22val%22%3A%2257ea5dd6-4c35-4982-942f-8a7f8b8c3a4b%22%2C%22options%22%3A%7B%22end%22%3A%222023-02-17T05%3A02%3A59.936Z%22%2C%22path%22%3A%22%2F%22%7D%7D; atidvisitor=%7B%22name%22%3A%22atidvisitor%22%2C%22val%22%3A%7B%22vrn%22%3A%22-595936-%22%7D%2C%22options%22%3A%7B%22path%22%3A%22%2F%22%2C%22session%22%3A15724800%2C%22end%22%3A15724800%7D%7D",
        "sec-ch-ua-mobile": "?0",
        "sec-ch-ua-platform": "Windows",
        "sec-fetch-dest": "document",
        "sec-fetch-mode": "navigate",
        "sec-fetch-site": "none",
        "sec-fetch-user": "?1",
        "upgrade-insecure-requests": "1"
    }

In [7]:
def saveString(html, filename="test.html"):
    try:
        file = open(filename,"w")
        file.write(str(html))
        file.close()
    except Exception as ex:
        print('Error: ' + str(ex))

In [8]:
base_url = "https://www.yelp.com/search?find_desc=Donut+Shop&find_loc=San+Francisco%2C+CA&ns=1"

for page_num, donut_num in enumerate(range(0, 40, 10), start=1):
    url = base_url + "&start=" + str(donut_num)
    page = requests.get(url ,headers=headers)
    book_productpage = BeautifulSoup(page.text, 'lxml')
    name = "sf_donut_shop_search_page_" + str(page_num) + ".html"
    saveString(book_productpage,filename=name)
    
    time.sleep(10)

## Question 2

In [3]:
shops = []
for page_num in range(1, 5):
    name = "sf_donut_shop_search_page_" + str(page_num) + ".html"
    f =codecs.open(name, 'r', 'utf-8')
    soup = BeautifulSoup(f.read(), 'lxml')
    
    name_tags = soup.find_all('div', attrs={'class': re.compile('^businessName.*')})
    list_items = [i.find_parent('li') for i in name_tags if re.match(r"^[0-9]+\.\s", i.text)]

    for item in list_items:
        shop_info = {}
        shop_info["Rank"] = item.select('h3')[0].find(text=True)
        shop_info["Name"] = item.select('h3 > span > a')[0].find(text=True)
        shop_info["Link"] = f"https://www.yelp.com{item.select('h3 > span > a')[0].get('href')}"
        shop_info["Rating"] = item.find_all('div', attrs={'class': re.compile('^i-stars.*')})[0].get('aria-label')
        shop_info["Rating Count"] = item.find_all('span', attrs={'class': re.compile('^reviewCount.*')})[0].text
        shop_info["Store Tags"] = [tag.text for tag in item.select('a > button > p')]
        
        price_element = item.find_all('span', attrs={'class': re.compile('^priceRange.*')})
        if len(price_element) > 0:
            shop_info["Price Range"] = price_element[0].text
        else:
            shop_info["Price Range"] = None
            
        order_element = item.find_all('span', text=re.compile("Start Order"))
        if len(order_element) > 0:
            shop_info["Order Button"] = True
        else:
            shop_info["Order Button"] = False
            
        service_tags = item.find_all('div', attrs={'class': re.compile('^border-color.*'), 'data-testid': re.compile('TRUSTED_PROPERTY')})
        if len(service_tags) > 0:
            shop_info["Service Tags"] = [tag.text for tag in service_tags[0].find_all('div') if tag.text != ""]
        else:
            shop_info["Service Tags"] = []

        shops.append(shop_info)
pprint(shops)

[{'Link': 'https://www.yelp.com/biz/bobs-donuts-and-pastry-shop-san-francisco?osq=Donut+Shop',
  'Name': 'Bob’s Donuts & Pastry Shop',
  'Order Button': False,
  'Price Range': '$',
  'Rank': '1',
  'Rating': '4.5 star rating',
  'Rating Count': '3328',
  'Service Tags': [],
  'Store Tags': ['Bakeries', 'Donuts']},
 {'Link': 'https://www.yelp.com/biz/uncle-bennys-donut-and-bagel-san-francisco?osq=Donut+Shop',
  'Name': 'Uncle Benny’s Donut and Bagel',
  'Order Button': True,
  'Price Range': '$$',
  'Rank': '2',
  'Rating': '4.5 star rating',
  'Rating Count': '389',
  'Service Tags': ['Outdoor dining', 'Delivery', 'Takeout'],
  'Store Tags': ['Donuts', 'Bagels', 'Coffee & Tea']},
 {'Link': 'https://www.yelp.com/biz/twisted-donuts-and-coffee-san-francisco?osq=Donut+Shop',
  'Name': 'Twisted Donuts and Coffee',
  'Order Button': False,
  'Price Range': '$',
  'Rank': '3',
  'Rating': '4.5 star rating',
  'Rating Count': '508',
  'Service Tags': ['Delivery', 'Takeout'],
  'Store Tags': [

## Question 3

In [4]:
client = MongoClient(port=27017)
db = client["ddr"]
col = db["donut_shops"]
col.insert_many(shops)

## Question 4

In [24]:
for shop in shops:
    url = shop['Link']
    page = requests.get(url ,headers=headers)
    donut_productpage = BeautifulSoup(page.text, 'lxml')
    name = "sf_donut_shop_" + str(shop['Rank']) + ".html"
    saveString(donut_productpage,filename=name)
    
    time.sleep(10)

## Question 5 & Question 6

In [26]:
donut_detail = []

base_url = "http://api.positionstack.com/v1/"
API_KEY = "52fb6e9394e0eadb757e3ef26bb46ab9"

for page_num in range(1, 41):
    name = "sf_donut_shop_" + str(page_num) + ".html"
    f =codecs.open(name, 'r', 'utf-8')
    soup = BeautifulSoup(f.read(), 'lxml')

    shop_info = {}

    shop_info["Name"] = soup.select('h1')[0].text.replace("'", "’")
    
    phone_element = soup.find_all('p', text=re.compile("Phone number"))
    if len(phone_element) > 0:
        shop_info["Phone Number"] = phone_element[0].find_parent('div').find_all('p')[1].text
    else:
        shop_info["Phone Number"] = None
        
    shop_info["Address"] = soup.find_all('address')[0].get_text(separator=' ')
    
    website_element = soup.find_all('p', text=re.compile("Business website"))
    if len(website_element) > 0:
        shop_info["Website"] = website_element[0].find_parent('div').find_all('p')[1].text
    
    # Get Geolocation
    url = base_url + "forward?access_key=" + API_KEY + "&query=" + str(shop_info['Address'])
    r = requests.get(url ,headers=headers)
    r = r.json()["data"][0]
    shop_info["geolocation"] = (r["latitude"], r["longitude"])
    
    # Update database
    db.sf_donut_shops.find_one_and_update({'Name': shop_info['Name']}, {'$set': shop_info})

    donut_detail.append(shop_info)

pprint(donut_detail)

# Reset Index
db.sf_donut_shops.create_index("Rank")

[{'Address': '1621 Polk St San Francisco, CA 94109',
  'Name': 'Bob’s Donuts & Pastry Shop',
  'Phone Number': '(415) 776-3141',
  'Website': 'http://www.bobsdonutssf.com',
  'geolocation': (37.791852, -122.421276)},
 {'Address': '2049 Irving St San Francisco, CA 94122',
  'Name': 'Uncle Benny’s Donut and Bagel',
  'Phone Number': '(628) 219-0956',
  'geolocation': (37.763299, -122.480192)},
 {'Address': '1243 Noriega St San Francisco, CA 94122',
  'Name': 'Twisted Donuts and Coffee',
  'Phone Number': '(415) 463-5437',
  'Website': 'http://twisteddonutsf.com',
  'geolocation': (37.754095, -122.477233)},
 {'Address': '601 Baker St San Francisco, CA 94117',
  'Name': 'Bob’s Donuts',
  'Phone Number': '(415) 776-3141',
  'Website': 'https://www.bobsdonutssf.com',
  'geolocation': (37.776539, -122.441854)},
 {'Address': '901 Clement St San Francisco, CA 94118',
  'Name': 'All Star Donuts',
  'Phone Number': '(415) 221-9838',
  'geolocation': (37.782559, -122.468983)},
 {'Address': '1399 9